In [ ]:
import torch

# Example data (inputs and targets)
X, Y = torch.randn((100, 10), requires_grad=False), torch.randn(100, requires_grad=False)

In [ ]:
# Model parameters
W1 = torch.randn(2, 10, requires_grad=True)
W2 = torch.randn(2, requires_grad=True)
learning_rate = 0.0001
optimizer = torch.optim.SGD([W1, W2], lr=learning_rate)

loss_all = []


for _ in range(100):
    for i in range(Y.shape[0]):
        y_pred1 = torch.matmul(W1, X[i])
        y_pred2 = torch.matmul(W2, y_pred1)

        loss = (y_pred2 - Y[i]).pow(2).sum()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


        loss_all.append(loss.detach().cpu().item())

import matplotlib.pyplot as plt

plt.plot(loss_all)
loss_all[0], loss_all[-1]

In [ ]:
# Model parameters
W1 = torch.randn(2, 10, requires_grad=True)
W2 = torch.randn(2, requires_grad=True)
learning_rate = 0.0001
optimizer = torch.optim.SGD([W1, W2], lr=learning_rate)

loss_all = []

for _ in range(100):
    y_pred1_all = []
    for i in range(Y.shape[0]):
        y_pred1 = torch.matmul(W1, X[i])
        y_pred1_all.append(y_pred1)

    for idx, y_pred1 in enumerate(y_pred1_all):
        y_pred2 = torch.matmul(W2, y_pred1)

        loss = (y_pred2 - Y[idx]).pow(2).sum()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


        loss_all.append(loss.detach().cpu().item())

import matplotlib.pyplot as plt

plt.plot(loss_all)
loss_all[0], loss_all[-1]

In [ ]:
import torch

# Example data (inputs and targets)
x1, y1 = torch.randn(10, requires_grad=False), torch.randn(1, requires_grad=False)
x2, y2 = torch.randn(10, requires_grad=False), torch.randn(1, requires_grad=False)

# Model parameters
W = torch.randn(1, 10, requires_grad=True)
learning_rate = 0.001
optimizer = torch.optim.SGD([W], lr=learning_rate)
# print(W)

for _ in range(100):
    y_pred1 = torch.matmul(W, x1)
    y_pred2 = torch.matmul(W, x2)

    loss = (y_pred1 - y1).pow(2).sum()
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    # print(W)

    loss = (y_pred2 - y2).pow(2).sum()
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    # print(W)
    print(loss)


In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.utils import add_self_loops

# Step 2: Define the GNN Model


class GCN(torch.nn.Module):
    def __init__(self,num_node_features: int):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(num_node_features, 16)
        self.conv2 = GCNConv(16, 16)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = F.relu(self.conv1(x, edge_index))
        x = self.conv2(x, edge_index)

        return x

# Step 1: Generate a Synthetic Graph
# Node features
x = torch.tensor([[1, 0], [0, 1], [1, 1], [1, 0]], dtype=torch.float)
# Edges
edge_index = torch.tensor([[0, 1, 2, 3, 0, 2],
                           [1, 0, 3, 2, 2, 0]], dtype=torch.long)
# Edge labels for classification
edge_label = torch.tensor([0, 1, 0, 1, 1, 0], dtype=torch.long)

data = Data(x=x, edge_index=edge_index)

model = GCN(num_node_features=data.num_node_features)

# Edge classifier: A simple linear layer
# Assuming we concatenate two node embeddings
edge_classifier = torch.nn.Linear(16 * 2, 2)

optimizer = torch.optim.Adam(
    list(model.parameters()) + list(edge_classifier.parameters()), lr=0.01)

# Step 3: Training Loop
for epoch in range(100):
    node_embeddings = model(data)

    # node_embeddings.shape == torch.Size([4, 16])

    # Get embeddings of both source and target nodes of each edge
    source_embeddings = node_embeddings[data.edge_index[0]]
    target_embeddings = node_embeddings[data.edge_index[1]]

    # Edge classification: Here we concatenate source and target embeddings
    edge_embeddings = torch.cat([source_embeddings, target_embeddings], dim=1)
    edge_pred = edge_classifier(edge_embeddings)

    loss = F.cross_entropy(edge_pred, edge_label)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Calculate accuracy
    _, pred = torch.max(edge_pred, dim=1)
    correct = pred.eq(edge_label).sum().item()
    accuracy = correct / edge_label.size(0)

    if epoch % 10 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item()}, Accuracy: {accuracy}')

In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv

# Synthetic Graph Data Setup (including node features, edges, and edge labels)
x = torch.tensor([[1, 0], [0, 1], [1, 1], [1, 0]],
                 dtype=torch.float)  # Node features
edge_index = torch.tensor(
    [[0, 1, 2, 3, 0, 2], [1, 0, 3, 2, 2, 0]], dtype=torch.long
)  # Edges
edge_label = torch.tensor(
    [0, 1, 0, 1, 1, 0], dtype=torch.long
)  # Edge labels for classification

data = Data(x=x, edge_index=edge_index)


# EnhancedGCN Model Definition with Learnable Embeddings
class EnhancedGCN(torch.nn.Module):
    def __init__(self, num_nodes, num_node_features, num_embedding_features):
        super(EnhancedGCN, self).__init__()
        self.embeddings = torch.nn.Embedding(num_nodes, num_embedding_features)
        self.embeddings.weight.data.uniform_(0, 1)  # Initialize embeddings

        self.conv1 = GCNConv(num_embedding_features, 16)
        self.conv2 = GCNConv(16, 16)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        node_indices = torch.arange(
            0, x.size(0), dtype=torch.long, device=x.device)
        node_embeddings = self.embeddings(node_indices)

        # x = torch.cat([x, node_embeddings], dim=1)  # Concatenate original features with embeddings
        # x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv1(node_embeddings, edge_index))

        x = self.conv2(x, edge_index)

        return x


# Model and Optimizer Setup
num_nodes = data.num_nodes
num_node_features = data.num_features
num_embedding_features = 5  # Example additional embedding size
model = EnhancedGCN(num_nodes, num_node_features, num_embedding_features)

edge_classifier = torch.nn.Linear(
    16 * 2, 2
)  # Classifier for the concatenated node embeddings
optimizer = torch.optim.Adam(
    list(model.parameters()) + list(edge_classifier.parameters()), lr=0.01
)

# Training Loop with Accuracy Calculation for Edge Classification
for epoch in range(100):
    optimizer.zero_grad()
    node_embeddings = model(data)

    source_embeddings = node_embeddings[data.edge_index[0]]
    target_embeddings = node_embeddings[data.edge_index[1]]
    edge_embeddings = torch.cat([source_embeddings, target_embeddings], dim=1)

    edge_pred = edge_classifier(edge_embeddings)
    loss = F.cross_entropy(edge_pred, edge_label)
    loss.backward()
    optimizer.step()

    # Calculate and print accuracy
    _, pred = torch.max(edge_pred, dim=1)
    correct = pred.eq(edge_label).sum().item()
    accuracy = correct / edge_label.size(0)

    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item()}, Accuracy: {accuracy}")

In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data


class EnhancedGCNWithEdgeClassification(torch.nn.Module):
    def __init__(self, num_nodes, num_node_features, num_embedding_features, out_channels):
        super(EnhancedGCNWithEdgeClassification, self).__init__()
        self.embeddings = torch.nn.Embedding(num_nodes, num_embedding_features)
        self.embeddings.weight.data.uniform_(0, 1)  # Initialize embeddings

        self.conv1 = GCNConv(num_node_features + num_embedding_features, 16)
        self.conv2 = GCNConv(16, out_channels)

        # Instead of a separate edge classifier, incorporate it into the GNN model
        # Assuming binary classification for edges
        self.edge_classifier = torch.nn.Linear(out_channels * 2, 2)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        node_indices = torch.arange(
            0, x.size(0), dtype=torch.long, device=x.device)
        node_embeddings = self.embeddings(node_indices)
        # Concatenate original features with embeddings
        x = torch.cat([x, node_embeddings], dim=1)

        x = F.relu(self.conv1(x, edge_index))
        x = self.conv2(x, edge_index)

        # Now, for each edge, use the node embeddings to predict edge labels
        source_embeddings = x[edge_index[0]]
        target_embeddings = x[edge_index[1]]
        edge_embeddings = torch.cat(
            [source_embeddings, target_embeddings], dim=1)
        edge_pred = self.edge_classifier(edge_embeddings)

        return edge_pred


# Parameters and Data Initialization
num_nodes = data.num_nodes
num_node_features = data.num_features
num_embedding_features = 5  # Example additional embedding size
out_channels = 16  # Example size for the node embeddings used in edge predictions

model = EnhancedGCNWithEdgeClassification(
    num_nodes, num_node_features, num_embedding_features, out_channels)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Assume edge_index, x, and edge_label are defined as in the previous examples

data = Data(x=x, edge_index=edge_index)

# Training Loop with the Adjusted Model
for epoch in range(100):
    optimizer.zero_grad()
    edge_pred = model(data)  # The model now directly outputs edge predictions
    loss = F.cross_entropy(edge_pred, edge_label)
    loss.backward()
    optimizer.step()

    # Calculate and print accuracy
    _, pred = torch.max(edge_pred, dim=1)
    correct = pred.eq(edge_label).sum().item()
    accuracy = correct / edge_label.size(0)

    if epoch % 10 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item()}, Accuracy: {accuracy}')

In [ ]:
import numpy as np
import torch
from torch_geometric.data import Data, DataLoader
from torch_geometric.nn import GCNConv, global_mean_pool
import torch.nn.functional as F


# Define the GCN Model
class GCN(torch.nn.Module):
    def __init__(self, num_features, num_outputs):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(num_features, 16)
        self.conv2 = GCNConv(16, num_outputs)

    def forward(self, x, edge_index, batch):
        x = F.relu(self.conv1(x, edge_index))
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        # Pool node features within each graph to a single vector
        # x = global_mean_pool(x, batch)
        # return F.log_softmax(x, dim=1)
        return x


# Generate some example data
def create_random_graph_data(num_node_features, num_classes, min_nodes=5, max_nodes=20):
    num_nodes = np.random.randint(
        min_nodes, max_nodes + 1
    )  # Randomly choose the number of nodes
    edge_prob = 0.2  # Probability of edge creation between any two nodes
    edge_index = []
    for i in range(num_nodes):
        for j in range(num_nodes):
            if i != j and np.random.rand() < edge_prob:
                edge_index.append([i, j])
    edge_index = (
        torch.tensor(edge_index).t().contiguous()
    )  # Transpose and make contiguous
    x = torch.randn(num_nodes, num_node_features)  # Node features
    y = torch.randint(0, num_classes, (1,)).item()  # Graph-level label
    return Data(x=x, edge_index=edge_index, y=y)


num_graphs = 20

# Use the new function to create a dataset with more varied graph sizes
graphs = [create_random_graph_data(3, 2) for _ in range(num_graphs)]

# DataLoader for batching graphs
loader = DataLoader(graphs, batch_size=4, shuffle=True)

# Initialize the model
model = GCN(num_features=3, num_outputs=8)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Train the model
model.train()
for epoch in range(10):
    for batch in loader:
        optimizer.zero_grad()
        out = model(batch.x, batch.edge_index, batch.batch)

        print(out.shape)
        loss = F.nll_loss(out, batch.y)
        loss.backward()
        optimizer.step()
        print(f"Epoch {epoch}, Loss: {loss.item()}")

<function any(iterable, /)>

In [12]:
batch

DataBatch(x=[50, 3], edge_index=[2, 123], y=[4], batch=[50], ptr=[5])

In [21]:
batch.edge_index[:, 1:5]

tensor([[1, 2, 2, 2],
        [6, 0, 4, 8]])

In [ ]:
import torch

# Example tensor of size N by F
tensor = torch.randn(5, 3)  # N=5, F=3

# Generate random permutation indices
perm_indices = torch.randperm(tensor.size(0))  # Random permutation of indices along dimension 0 (rows)

# Permute rows of the tensor
permuted_tensor = torch.index_select(tensor, 0, perm_indices)

print("Original Tensor:")
print(tensor)
print("\nPermuted Tensor (Randomly Permuted Rows):")
print(permuted_tensor)


In [58]:
import torch

state = torch.randn(4, 5)  # State tensor


gnn_features = torch.randn(4, 2, 3)  # GNN output features
done = torch.Tensor([1, 0, 0, 0]).reshape(-1, 1)
gnn_features.shape, done.shape

(torch.Size([4, 2, 3]), torch.Size([4, 1]))

In [59]:
state.shape

torch.Size([4, 5])

In [50]:
# reshape gnn_features from (10, 5, 3) to (10*5, 3) in place
gnn_features_reshaped = gnn_features.view(-1, gnn_features.size(-1))
gnn_features_reshaped.shape


done_expanded = done.expand(gnn_features.shape[0], -1, -1)


In [52]:
done_expanded.shape

torch.Size([4, 4, 1])